In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

In [2]:
data_normal = pd.read_csv('Save_Normal_data.csv')
scaled_data_normal = pd.read_csv('Save_Normal_data_SCALED.csv')
scaled_fac = pd.read_csv('Normal_Scaled_Factor.csv')

In [3]:
def phaseSelect(value, let):
    result = 0
    if (let in value):
        result = 1
    else:
        result =0
    return result
dict_voltage= {"MediumVoltage_ShortLine":34641.0161513775
               , "HighVoltage_ShortLine":326598
               , "HighVoltage_LongLine300":326598
              , "HighVoltage_LongLine500":326598}
def VoltageSelect(typeLine):
    return dict_voltage[typeLine]
scaled_data_normal["Is_A"] = scaled_data_normal["phase"].apply(lambda x: phaseSelect(x,"A"))
scaled_data_normal["Is_B"] = scaled_data_normal["phase"].apply(lambda x: phaseSelect(x,"B"))
scaled_data_normal["Is_C"] = scaled_data_normal["phase"].apply(lambda x: phaseSelect(x,"C"))
scaled_type = pd.get_dummies(scaled_data_normal["Type"],drop_first=True)
scaled_data_normal = pd.concat([scaled_data_normal,scaled_type],axis=1)
data_normal["Is_A"] = data_normal["phase"].apply(lambda x: phaseSelect(x,"A"))
data_normal["Is_B"] = data_normal["phase"].apply(lambda x: phaseSelect(x,"B"))
data_normal["Is_C"] = data_normal["phase"].apply(lambda x: phaseSelect(x,"C"))
normal_type = pd.get_dummies(data_normal["Type"],drop_first=True)
data_normal = pd.concat([data_normal, scaled_type],axis=1)
#data_normal["NewCycle"] = data_normal["cycle"]/20
#scaled_data_normal["NewCycle"] = scaled_data_normal["cycle"]/20

data_normal["VoltageGenerator"] = data_normal["Type"].apply(lambda x: VoltageSelect(x))
scaled_data_normal["VoltageGenerator"] = scaled_data_normal["Type"].apply(lambda x: VoltageSelect(x))/max(dict_voltage.values())
scaled_data_normal["ScaledLength"] = scaled_data_normal["TotalLength"]/scaled_data_normal["TotalLength"].max()
data_normal["ScaledLength"] = data_normal["TotalLength"]/data_normal["TotalLength"].max()

# MACHINE LEARNING

In [4]:
all_column = list(scaled_data_normal.columns)
studied_columns_all = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','ScaledLength',
'Iabc_generator_ContinueComponent_PhaseA','Iabc_generator_ContinueComponent_PhaseB','Iabc_generator_ContinueComponent_PhaseC','Iabc_generator_EffectiveComponent_PhaseA','Iabc_generator_EffectiveComponent_PhaseB','Iabc_generator_EffectiveComponent_PhaseC','Iabc_generator_FirstArmonic_PhaseA_Value','Iabc_generator_FirstArmonic_PhaseA_Angle','Iabc_generator_FirstArmonic_PhaseB_Value','Iabc_generator_FirstArmonic_PhaseB_Angle','Iabc_generator_FirstArmonic_PhaseC_Value','Iabc_generator_FirstArmonic_PhaseC_Angle','Iabc_generator_DeformingComponent_PhaseA','Iabc_generator_DeformingComponent_PhaseB','Iabc_generator_DeformingComponent_PhaseC','Iabc_generator_CoefficientDistorsion_PhaseA','Iabc_generator_CoefficientDistorsion_PhaseB','Iabc_generator_CoefficientDistorsion_PhaseC','Iabc_generator_CoefficientDistorsionContinue_PhaseA','Iabc_generator_CoefficientDistorsionContinue_PhaseB','Iabc_generator_CoefficientDistorsionContinue_PhaseC','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseA','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseB','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseC','Iabc_generator_DirectComponent_Value','Iabc_generator_DirectComponent_Angle','Iabc_generator_InverseComponent_Value','Iabc_generator_InverseComponent_Angle','Iabc_generator_ZeroComponent_Value','Iabc_generator_ZeroComponent_Angle','Iabc_generator_CoefficeintNegativeAsimetry','Iabc_generator_CoefficeintZeroAsimetry',
'Vabc_generator_ContinueComponent_PhaseA','Vabc_generator_ContinueComponent_PhaseB','Vabc_generator_ContinueComponent_PhaseC','Vabc_generator_EffectiveComponent_PhaseA','Vabc_generator_EffectiveComponent_PhaseB','Vabc_generator_EffectiveComponent_PhaseC','Vabc_generator_FirstArmonic_PhaseA_Value','Vabc_generator_FirstArmonic_PhaseA_Angle','Vabc_generator_FirstArmonic_PhaseB_Value','Vabc_generator_FirstArmonic_PhaseB_Angle','Vabc_generator_FirstArmonic_PhaseC_Value','Vabc_generator_FirstArmonic_PhaseC_Angle','Vabc_generator_DeformingComponent_PhaseA','Vabc_generator_DeformingComponent_PhaseB','Vabc_generator_DeformingComponent_PhaseC','Vabc_generator_CoefficientDistorsion_PhaseA','Vabc_generator_CoefficientDistorsion_PhaseB','Vabc_generator_CoefficientDistorsion_PhaseC','Vabc_generator_CoefficientDistorsionContinue_PhaseA','Vabc_generator_CoefficientDistorsionContinue_PhaseB','Vabc_generator_CoefficientDistorsionContinue_PhaseC','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseA','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseB','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseC','Vabc_generator_DirectComponent_Value','Vabc_generator_DirectComponent_Angle','Vabc_generator_InverseComponent_Value','Vabc_generator_InverseComponent_Angle','Vabc_generator_ZeroComponent_Value','Vabc_generator_ZeroComponent_Angle','Vabc_generator_CoefficeintNegativeAsimetry','Vabc_generator_CoefficeintZeroAsimetry',
'Iabc_consumator_ContinueComponent_PhaseA','Iabc_consumator_ContinueComponent_PhaseB','Iabc_consumator_ContinueComponent_PhaseC','Iabc_consumator_EffectiveComponent_PhaseA','Iabc_consumator_EffectiveComponent_PhaseB','Iabc_consumator_EffectiveComponent_PhaseC','Iabc_consumator_FirstArmonic_PhaseA_Value','Iabc_consumator_FirstArmonic_PhaseA_Angle','Iabc_consumator_FirstArmonic_PhaseB_Value','Iabc_consumator_FirstArmonic_PhaseB_Angle','Iabc_consumator_FirstArmonic_PhaseC_Value','Iabc_consumator_FirstArmonic_PhaseC_Angle','Iabc_consumator_DeformingComponent_PhaseA','Iabc_consumator_DeformingComponent_PhaseB','Iabc_consumator_DeformingComponent_PhaseC','Iabc_consumator_CoefficientDistorsion_PhaseA','Iabc_consumator_CoefficientDistorsion_PhaseB','Iabc_consumator_CoefficientDistorsion_PhaseC','Iabc_consumator_CoefficientDistorsionContinue_PhaseA','Iabc_consumator_CoefficientDistorsionContinue_PhaseB','Iabc_consumator_CoefficientDistorsionContinue_PhaseC','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC','Iabc_consumator_DirectComponent_Value','Iabc_consumator_DirectComponent_Angle','Iabc_consumator_InverseComponent_Value','Iabc_consumator_InverseComponent_Angle','Iabc_consumator_ZeroComponent_Value','Iabc_consumator_ZeroComponent_Angle','Iabc_consumator_CoefficeintNegativeAsimetry','Iabc_consumator_CoefficeintZeroAsimetry',
'Vabc_consumator_ContinueComponent_PhaseA','Vabc_consumator_ContinueComponent_PhaseB','Vabc_consumator_ContinueComponent_PhaseC','Vabc_consumator_EffectiveComponent_PhaseA','Vabc_consumator_EffectiveComponent_PhaseB','Vabc_consumator_EffectiveComponent_PhaseC','Vabc_consumator_FirstArmonic_PhaseA_Value','Vabc_consumator_FirstArmonic_PhaseA_Angle','Vabc_consumator_FirstArmonic_PhaseB_Value','Vabc_consumator_FirstArmonic_PhaseB_Angle','Vabc_consumator_FirstArmonic_PhaseC_Value','Vabc_consumator_FirstArmonic_PhaseC_Angle','Vabc_consumator_DeformingComponent_PhaseA','Vabc_consumator_DeformingComponent_PhaseB','Vabc_consumator_DeformingComponent_PhaseC','Vabc_consumator_CoefficientDistorsion_PhaseA','Vabc_consumator_CoefficientDistorsion_PhaseB','Vabc_consumator_CoefficientDistorsion_PhaseC','Vabc_consumator_CoefficientDistorsionContinue_PhaseA','Vabc_consumator_CoefficientDistorsionContinue_PhaseB','Vabc_consumator_CoefficientDistorsionContinue_PhaseC','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC','Vabc_consumator_DirectComponent_Value','Vabc_consumator_DirectComponent_Angle','Vabc_consumator_InverseComponent_Value','Vabc_consumator_InverseComponent_Angle','Vabc_consumator_ZeroComponent_Value','Vabc_consumator_ZeroComponent_Angle','Vabc_consumator_CoefficeintNegativeAsimetry','Vabc_consumator_CoefficeintZeroAsimetry'
]

studied_columns_generator = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','ScaledLength',
'Iabc_generator_ContinueComponent_PhaseA','Iabc_generator_ContinueComponent_PhaseB','Iabc_generator_ContinueComponent_PhaseC','Iabc_generator_EffectiveComponent_PhaseA','Iabc_generator_EffectiveComponent_PhaseB','Iabc_generator_EffectiveComponent_PhaseC','Iabc_generator_FirstArmonic_PhaseA_Value','Iabc_generator_FirstArmonic_PhaseA_Angle','Iabc_generator_FirstArmonic_PhaseB_Value','Iabc_generator_FirstArmonic_PhaseB_Angle','Iabc_generator_FirstArmonic_PhaseC_Value','Iabc_generator_FirstArmonic_PhaseC_Angle','Iabc_generator_DeformingComponent_PhaseA','Iabc_generator_DeformingComponent_PhaseB','Iabc_generator_DeformingComponent_PhaseC','Iabc_generator_CoefficientDistorsion_PhaseA','Iabc_generator_CoefficientDistorsion_PhaseB','Iabc_generator_CoefficientDistorsion_PhaseC','Iabc_generator_CoefficientDistorsionContinue_PhaseA','Iabc_generator_CoefficientDistorsionContinue_PhaseB','Iabc_generator_CoefficientDistorsionContinue_PhaseC','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseA','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseB','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseC','Iabc_generator_DirectComponent_Value','Iabc_generator_DirectComponent_Angle','Iabc_generator_InverseComponent_Value','Iabc_generator_InverseComponent_Angle','Iabc_generator_ZeroComponent_Value','Iabc_generator_ZeroComponent_Angle','Iabc_generator_CoefficeintNegativeAsimetry','Iabc_generator_CoefficeintZeroAsimetry',
'Vabc_generator_ContinueComponent_PhaseA','Vabc_generator_ContinueComponent_PhaseB','Vabc_generator_ContinueComponent_PhaseC','Vabc_generator_EffectiveComponent_PhaseA','Vabc_generator_EffectiveComponent_PhaseB','Vabc_generator_EffectiveComponent_PhaseC','Vabc_generator_FirstArmonic_PhaseA_Value','Vabc_generator_FirstArmonic_PhaseA_Angle','Vabc_generator_FirstArmonic_PhaseB_Value','Vabc_generator_FirstArmonic_PhaseB_Angle','Vabc_generator_FirstArmonic_PhaseC_Value','Vabc_generator_FirstArmonic_PhaseC_Angle','Vabc_generator_DeformingComponent_PhaseA','Vabc_generator_DeformingComponent_PhaseB','Vabc_generator_DeformingComponent_PhaseC','Vabc_generator_CoefficientDistorsion_PhaseA','Vabc_generator_CoefficientDistorsion_PhaseB','Vabc_generator_CoefficientDistorsion_PhaseC','Vabc_generator_CoefficientDistorsionContinue_PhaseA','Vabc_generator_CoefficientDistorsionContinue_PhaseB','Vabc_generator_CoefficientDistorsionContinue_PhaseC','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseA','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseB','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseC','Vabc_generator_DirectComponent_Value','Vabc_generator_DirectComponent_Angle','Vabc_generator_InverseComponent_Value','Vabc_generator_InverseComponent_Angle','Vabc_generator_ZeroComponent_Value','Vabc_generator_ZeroComponent_Angle','Vabc_generator_CoefficeintNegativeAsimetry','Vabc_generator_CoefficeintZeroAsimetry'
]

studied_columns_consumator = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','ScaledLength',
'Iabc_consumator_ContinueComponent_PhaseA','Iabc_consumator_ContinueComponent_PhaseB','Iabc_consumator_ContinueComponent_PhaseC','Iabc_consumator_EffectiveComponent_PhaseA','Iabc_consumator_EffectiveComponent_PhaseB','Iabc_consumator_EffectiveComponent_PhaseC','Iabc_consumator_FirstArmonic_PhaseA_Value','Iabc_consumator_FirstArmonic_PhaseA_Angle','Iabc_consumator_FirstArmonic_PhaseB_Value','Iabc_consumator_FirstArmonic_PhaseB_Angle','Iabc_consumator_FirstArmonic_PhaseC_Value','Iabc_consumator_FirstArmonic_PhaseC_Angle','Iabc_consumator_DeformingComponent_PhaseA','Iabc_consumator_DeformingComponent_PhaseB','Iabc_consumator_DeformingComponent_PhaseC','Iabc_consumator_CoefficientDistorsion_PhaseA','Iabc_consumator_CoefficientDistorsion_PhaseB','Iabc_consumator_CoefficientDistorsion_PhaseC','Iabc_consumator_CoefficientDistorsionContinue_PhaseA','Iabc_consumator_CoefficientDistorsionContinue_PhaseB','Iabc_consumator_CoefficientDistorsionContinue_PhaseC','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC','Iabc_consumator_DirectComponent_Value','Iabc_consumator_DirectComponent_Angle','Iabc_consumator_InverseComponent_Value','Iabc_consumator_InverseComponent_Angle','Iabc_consumator_ZeroComponent_Value','Iabc_consumator_ZeroComponent_Angle','Iabc_consumator_CoefficeintNegativeAsimetry','Iabc_consumator_CoefficeintZeroAsimetry',
'Vabc_consumator_ContinueComponent_PhaseA','Vabc_consumator_ContinueComponent_PhaseB','Vabc_consumator_ContinueComponent_PhaseC','Vabc_consumator_EffectiveComponent_PhaseA','Vabc_consumator_EffectiveComponent_PhaseB','Vabc_consumator_EffectiveComponent_PhaseC','Vabc_consumator_FirstArmonic_PhaseA_Value','Vabc_consumator_FirstArmonic_PhaseA_Angle','Vabc_consumator_FirstArmonic_PhaseB_Value','Vabc_consumator_FirstArmonic_PhaseB_Angle','Vabc_consumator_FirstArmonic_PhaseC_Value','Vabc_consumator_FirstArmonic_PhaseC_Angle','Vabc_consumator_DeformingComponent_PhaseA','Vabc_consumator_DeformingComponent_PhaseB','Vabc_consumator_DeformingComponent_PhaseC','Vabc_consumator_CoefficientDistorsion_PhaseA','Vabc_consumator_CoefficientDistorsion_PhaseB','Vabc_consumator_CoefficientDistorsion_PhaseC','Vabc_consumator_CoefficientDistorsionContinue_PhaseA','Vabc_consumator_CoefficientDistorsionContinue_PhaseB','Vabc_consumator_CoefficientDistorsionContinue_PhaseC','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC','Vabc_consumator_DirectComponent_Value','Vabc_consumator_DirectComponent_Angle','Vabc_consumator_InverseComponent_Value','Vabc_consumator_InverseComponent_Angle','Vabc_consumator_ZeroComponent_Value','Vabc_consumator_ZeroComponent_Angle','Vabc_consumator_CoefficeintNegativeAsimetry','Vabc_consumator_CoefficeintZeroAsimetry'
]
all_column_normal = list(data_normal.columns)
studied_columns_all_normal = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','TotalLength',
'Iabc_generator_ContinueComponent_PhaseA','Iabc_generator_ContinueComponent_PhaseB','Iabc_generator_ContinueComponent_PhaseC','Iabc_generator_EffectiveComponent_PhaseA','Iabc_generator_EffectiveComponent_PhaseB','Iabc_generator_EffectiveComponent_PhaseC','Iabc_generator_FirstArmonic_PhaseA_Value','Iabc_generator_FirstArmonic_PhaseA_Angle','Iabc_generator_FirstArmonic_PhaseB_Value','Iabc_generator_FirstArmonic_PhaseB_Angle','Iabc_generator_FirstArmonic_PhaseC_Value','Iabc_generator_FirstArmonic_PhaseC_Angle','Iabc_generator_DeformingComponent_PhaseA','Iabc_generator_DeformingComponent_PhaseB','Iabc_generator_DeformingComponent_PhaseC','Iabc_generator_CoefficientDistorsion_PhaseA','Iabc_generator_CoefficientDistorsion_PhaseB','Iabc_generator_CoefficientDistorsion_PhaseC','Iabc_generator_CoefficientDistorsionContinue_PhaseA','Iabc_generator_CoefficientDistorsionContinue_PhaseB','Iabc_generator_CoefficientDistorsionContinue_PhaseC','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseA','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseB','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseC','Iabc_generator_DirectComponent_Value','Iabc_generator_DirectComponent_Angle','Iabc_generator_InverseComponent_Value','Iabc_generator_InverseComponent_Angle','Iabc_generator_ZeroComponent_Value','Iabc_generator_ZeroComponent_Angle','Iabc_generator_CoefficeintNegativeAsimetry','Iabc_generator_CoefficeintZeroAsimetry',
'Vabc_generator_ContinueComponent_PhaseA','Vabc_generator_ContinueComponent_PhaseB','Vabc_generator_ContinueComponent_PhaseC','Vabc_generator_EffectiveComponent_PhaseA','Vabc_generator_EffectiveComponent_PhaseB','Vabc_generator_EffectiveComponent_PhaseC','Vabc_generator_FirstArmonic_PhaseA_Value','Vabc_generator_FirstArmonic_PhaseA_Angle','Vabc_generator_FirstArmonic_PhaseB_Value','Vabc_generator_FirstArmonic_PhaseB_Angle','Vabc_generator_FirstArmonic_PhaseC_Value','Vabc_generator_FirstArmonic_PhaseC_Angle','Vabc_generator_DeformingComponent_PhaseA','Vabc_generator_DeformingComponent_PhaseB','Vabc_generator_DeformingComponent_PhaseC','Vabc_generator_CoefficientDistorsion_PhaseA','Vabc_generator_CoefficientDistorsion_PhaseB','Vabc_generator_CoefficientDistorsion_PhaseC','Vabc_generator_CoefficientDistorsionContinue_PhaseA','Vabc_generator_CoefficientDistorsionContinue_PhaseB','Vabc_generator_CoefficientDistorsionContinue_PhaseC','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseA','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseB','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseC','Vabc_generator_DirectComponent_Value','Vabc_generator_DirectComponent_Angle','Vabc_generator_InverseComponent_Value','Vabc_generator_InverseComponent_Angle','Vabc_generator_ZeroComponent_Value','Vabc_generator_ZeroComponent_Angle','Vabc_generator_CoefficeintNegativeAsimetry','Vabc_generator_CoefficeintZeroAsimetry',
'Iabc_consumator_ContinueComponent_PhaseA','Iabc_consumator_ContinueComponent_PhaseB','Iabc_consumator_ContinueComponent_PhaseC','Iabc_consumator_EffectiveComponent_PhaseA','Iabc_consumator_EffectiveComponent_PhaseB','Iabc_consumator_EffectiveComponent_PhaseC','Iabc_consumator_FirstArmonic_PhaseA_Value','Iabc_consumator_FirstArmonic_PhaseA_Angle','Iabc_consumator_FirstArmonic_PhaseB_Value','Iabc_consumator_FirstArmonic_PhaseB_Angle','Iabc_consumator_FirstArmonic_PhaseC_Value','Iabc_consumator_FirstArmonic_PhaseC_Angle','Iabc_consumator_DeformingComponent_PhaseA','Iabc_consumator_DeformingComponent_PhaseB','Iabc_consumator_DeformingComponent_PhaseC','Iabc_consumator_CoefficientDistorsion_PhaseA','Iabc_consumator_CoefficientDistorsion_PhaseB','Iabc_consumator_CoefficientDistorsion_PhaseC','Iabc_consumator_CoefficientDistorsionContinue_PhaseA','Iabc_consumator_CoefficientDistorsionContinue_PhaseB','Iabc_consumator_CoefficientDistorsionContinue_PhaseC','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC','Iabc_consumator_DirectComponent_Value','Iabc_consumator_DirectComponent_Angle','Iabc_consumator_InverseComponent_Value','Iabc_consumator_InverseComponent_Angle','Iabc_consumator_ZeroComponent_Value','Iabc_consumator_ZeroComponent_Angle','Iabc_consumator_CoefficeintNegativeAsimetry','Iabc_consumator_CoefficeintZeroAsimetry',
'Vabc_consumator_ContinueComponent_PhaseA','Vabc_consumator_ContinueComponent_PhaseB','Vabc_consumator_ContinueComponent_PhaseC','Vabc_consumator_EffectiveComponent_PhaseA','Vabc_consumator_EffectiveComponent_PhaseB','Vabc_consumator_EffectiveComponent_PhaseC','Vabc_consumator_FirstArmonic_PhaseA_Value','Vabc_consumator_FirstArmonic_PhaseA_Angle','Vabc_consumator_FirstArmonic_PhaseB_Value','Vabc_consumator_FirstArmonic_PhaseB_Angle','Vabc_consumator_FirstArmonic_PhaseC_Value','Vabc_consumator_FirstArmonic_PhaseC_Angle','Vabc_consumator_DeformingComponent_PhaseA','Vabc_consumator_DeformingComponent_PhaseB','Vabc_consumator_DeformingComponent_PhaseC','Vabc_consumator_CoefficientDistorsion_PhaseA','Vabc_consumator_CoefficientDistorsion_PhaseB','Vabc_consumator_CoefficientDistorsion_PhaseC','Vabc_consumator_CoefficientDistorsionContinue_PhaseA','Vabc_consumator_CoefficientDistorsionContinue_PhaseB','Vabc_consumator_CoefficientDistorsionContinue_PhaseC','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC','Vabc_consumator_DirectComponent_Value','Vabc_consumator_DirectComponent_Angle','Vabc_consumator_InverseComponent_Value','Vabc_consumator_InverseComponent_Angle','Vabc_consumator_ZeroComponent_Value','Vabc_consumator_ZeroComponent_Angle','Vabc_consumator_CoefficeintNegativeAsimetry','Vabc_consumator_CoefficeintZeroAsimetry'
]

studied_columns_generator_normal = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','TotalLength',
'Iabc_generator_ContinueComponent_PhaseA','Iabc_generator_ContinueComponent_PhaseB','Iabc_generator_ContinueComponent_PhaseC','Iabc_generator_EffectiveComponent_PhaseA','Iabc_generator_EffectiveComponent_PhaseB','Iabc_generator_EffectiveComponent_PhaseC','Iabc_generator_FirstArmonic_PhaseA_Value','Iabc_generator_FirstArmonic_PhaseA_Angle','Iabc_generator_FirstArmonic_PhaseB_Value','Iabc_generator_FirstArmonic_PhaseB_Angle','Iabc_generator_FirstArmonic_PhaseC_Value','Iabc_generator_FirstArmonic_PhaseC_Angle','Iabc_generator_DeformingComponent_PhaseA','Iabc_generator_DeformingComponent_PhaseB','Iabc_generator_DeformingComponent_PhaseC','Iabc_generator_CoefficientDistorsion_PhaseA','Iabc_generator_CoefficientDistorsion_PhaseB','Iabc_generator_CoefficientDistorsion_PhaseC','Iabc_generator_CoefficientDistorsionContinue_PhaseA','Iabc_generator_CoefficientDistorsionContinue_PhaseB','Iabc_generator_CoefficientDistorsionContinue_PhaseC','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseA','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseB','Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseC','Iabc_generator_DirectComponent_Value','Iabc_generator_DirectComponent_Angle','Iabc_generator_InverseComponent_Value','Iabc_generator_InverseComponent_Angle','Iabc_generator_ZeroComponent_Value','Iabc_generator_ZeroComponent_Angle','Iabc_generator_CoefficeintNegativeAsimetry','Iabc_generator_CoefficeintZeroAsimetry',
'Vabc_generator_ContinueComponent_PhaseA','Vabc_generator_ContinueComponent_PhaseB','Vabc_generator_ContinueComponent_PhaseC','Vabc_generator_EffectiveComponent_PhaseA','Vabc_generator_EffectiveComponent_PhaseB','Vabc_generator_EffectiveComponent_PhaseC','Vabc_generator_FirstArmonic_PhaseA_Value','Vabc_generator_FirstArmonic_PhaseA_Angle','Vabc_generator_FirstArmonic_PhaseB_Value','Vabc_generator_FirstArmonic_PhaseB_Angle','Vabc_generator_FirstArmonic_PhaseC_Value','Vabc_generator_FirstArmonic_PhaseC_Angle','Vabc_generator_DeformingComponent_PhaseA','Vabc_generator_DeformingComponent_PhaseB','Vabc_generator_DeformingComponent_PhaseC','Vabc_generator_CoefficientDistorsion_PhaseA','Vabc_generator_CoefficientDistorsion_PhaseB','Vabc_generator_CoefficientDistorsion_PhaseC','Vabc_generator_CoefficientDistorsionContinue_PhaseA','Vabc_generator_CoefficientDistorsionContinue_PhaseB','Vabc_generator_CoefficientDistorsionContinue_PhaseC','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseA','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseB','Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseC','Vabc_generator_DirectComponent_Value','Vabc_generator_DirectComponent_Angle','Vabc_generator_InverseComponent_Value','Vabc_generator_InverseComponent_Angle','Vabc_generator_ZeroComponent_Value','Vabc_generator_ZeroComponent_Angle','Vabc_generator_CoefficeintNegativeAsimetry','Vabc_generator_CoefficeintZeroAsimetry'
]

studied_columns_consumator_normal = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','TotalLength',
'Iabc_consumator_ContinueComponent_PhaseA','Iabc_consumator_ContinueComponent_PhaseB','Iabc_consumator_ContinueComponent_PhaseC','Iabc_consumator_EffectiveComponent_PhaseA','Iabc_consumator_EffectiveComponent_PhaseB','Iabc_consumator_EffectiveComponent_PhaseC','Iabc_consumator_FirstArmonic_PhaseA_Value','Iabc_consumator_FirstArmonic_PhaseA_Angle','Iabc_consumator_FirstArmonic_PhaseB_Value','Iabc_consumator_FirstArmonic_PhaseB_Angle','Iabc_consumator_FirstArmonic_PhaseC_Value','Iabc_consumator_FirstArmonic_PhaseC_Angle','Iabc_consumator_DeformingComponent_PhaseA','Iabc_consumator_DeformingComponent_PhaseB','Iabc_consumator_DeformingComponent_PhaseC','Iabc_consumator_CoefficientDistorsion_PhaseA','Iabc_consumator_CoefficientDistorsion_PhaseB','Iabc_consumator_CoefficientDistorsion_PhaseC','Iabc_consumator_CoefficientDistorsionContinue_PhaseA','Iabc_consumator_CoefficientDistorsionContinue_PhaseB','Iabc_consumator_CoefficientDistorsionContinue_PhaseC','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB','Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC','Iabc_consumator_DirectComponent_Value','Iabc_consumator_DirectComponent_Angle','Iabc_consumator_InverseComponent_Value','Iabc_consumator_InverseComponent_Angle','Iabc_consumator_ZeroComponent_Value','Iabc_consumator_ZeroComponent_Angle','Iabc_consumator_CoefficeintNegativeAsimetry','Iabc_consumator_CoefficeintZeroAsimetry',
'Vabc_consumator_ContinueComponent_PhaseA','Vabc_consumator_ContinueComponent_PhaseB','Vabc_consumator_ContinueComponent_PhaseC','Vabc_consumator_EffectiveComponent_PhaseA','Vabc_consumator_EffectiveComponent_PhaseB','Vabc_consumator_EffectiveComponent_PhaseC','Vabc_consumator_FirstArmonic_PhaseA_Value','Vabc_consumator_FirstArmonic_PhaseA_Angle','Vabc_consumator_FirstArmonic_PhaseB_Value','Vabc_consumator_FirstArmonic_PhaseB_Angle','Vabc_consumator_FirstArmonic_PhaseC_Value','Vabc_consumator_FirstArmonic_PhaseC_Angle','Vabc_consumator_DeformingComponent_PhaseA','Vabc_consumator_DeformingComponent_PhaseB','Vabc_consumator_DeformingComponent_PhaseC','Vabc_consumator_CoefficientDistorsion_PhaseA','Vabc_consumator_CoefficientDistorsion_PhaseB','Vabc_consumator_CoefficientDistorsion_PhaseC','Vabc_consumator_CoefficientDistorsionContinue_PhaseA','Vabc_consumator_CoefficientDistorsionContinue_PhaseB','Vabc_consumator_CoefficientDistorsionContinue_PhaseC','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB','Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC','Vabc_consumator_DirectComponent_Value','Vabc_consumator_DirectComponent_Angle','Vabc_consumator_InverseComponent_Value','Vabc_consumator_InverseComponent_Angle','Vabc_consumator_ZeroComponent_Value','Vabc_consumator_ZeroComponent_Angle','Vabc_consumator_CoefficeintNegativeAsimetry','Vabc_consumator_CoefficeintZeroAsimetry'
]
studied_columns = studied_columns_all

In [5]:
firstArmonic_columns_all_normal = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','TotalLength',
'Iabc_generator_FirstArmonic_PhaseA_Value','Iabc_generator_FirstArmonic_PhaseA_Angle','Iabc_generator_FirstArmonic_PhaseB_Value','Iabc_generator_FirstArmonic_PhaseB_Angle','Iabc_generator_FirstArmonic_PhaseC_Value','Iabc_generator_FirstArmonic_PhaseC_Angle',
'Vabc_generator_FirstArmonic_PhaseA_Value','Vabc_generator_FirstArmonic_PhaseA_Angle','Vabc_generator_FirstArmonic_PhaseB_Value','Vabc_generator_FirstArmonic_PhaseB_Angle','Vabc_generator_FirstArmonic_PhaseC_Value','Vabc_generator_FirstArmonic_PhaseC_Angle',
'Iabc_consumator_FirstArmonic_PhaseA_Value','Iabc_consumator_FirstArmonic_PhaseA_Angle','Iabc_consumator_FirstArmonic_PhaseB_Value','Iabc_consumator_FirstArmonic_PhaseB_Angle','Iabc_consumator_FirstArmonic_PhaseC_Value','Iabc_consumator_FirstArmonic_PhaseC_Angle',
'Vabc_consumator_FirstArmonic_PhaseA_Value','Vabc_consumator_FirstArmonic_PhaseA_Angle','Vabc_consumator_FirstArmonic_PhaseB_Value','Vabc_consumator_FirstArmonic_PhaseB_Angle','Vabc_consumator_FirstArmonic_PhaseC_Value','Vabc_consumator_FirstArmonic_PhaseC_Angle'
]
firstArmonic_columns_all = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','ScaledLength',
'Iabc_generator_FirstArmonic_PhaseA_Value','Iabc_generator_FirstArmonic_PhaseA_Angle','Iabc_generator_FirstArmonic_PhaseB_Value','Iabc_generator_FirstArmonic_PhaseB_Angle','Iabc_generator_FirstArmonic_PhaseC_Value','Iabc_generator_FirstArmonic_PhaseC_Angle',
'Vabc_generator_FirstArmonic_PhaseA_Value','Vabc_generator_FirstArmonic_PhaseA_Angle','Vabc_generator_FirstArmonic_PhaseB_Value','Vabc_generator_FirstArmonic_PhaseB_Angle','Vabc_generator_FirstArmonic_PhaseC_Value','Vabc_generator_FirstArmonic_PhaseC_Angle',
'Iabc_consumator_FirstArmonic_PhaseA_Value','Iabc_consumator_FirstArmonic_PhaseA_Angle','Iabc_consumator_FirstArmonic_PhaseB_Value','Iabc_consumator_FirstArmonic_PhaseB_Angle','Iabc_consumator_FirstArmonic_PhaseC_Value','Iabc_consumator_FirstArmonic_PhaseC_Angle',
'Vabc_consumator_FirstArmonic_PhaseA_Value','Vabc_consumator_FirstArmonic_PhaseA_Angle','Vabc_consumator_FirstArmonic_PhaseB_Value','Vabc_consumator_FirstArmonic_PhaseB_Angle','Vabc_consumator_FirstArmonic_PhaseC_Value','Vabc_consumator_FirstArmonic_PhaseC_Angle'
]
firstArmonic_columns_generator_normal = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','TotalLength',
'Iabc_generator_FirstArmonic_PhaseA_Value','Iabc_generator_FirstArmonic_PhaseA_Angle','Iabc_generator_FirstArmonic_PhaseB_Value','Iabc_generator_FirstArmonic_PhaseB_Angle','Iabc_generator_FirstArmonic_PhaseC_Value','Iabc_generator_FirstArmonic_PhaseC_Angle',
'Vabc_generator_FirstArmonic_PhaseA_Value','Vabc_generator_FirstArmonic_PhaseA_Angle','Vabc_generator_FirstArmonic_PhaseB_Value','Vabc_generator_FirstArmonic_PhaseB_Angle','Vabc_generator_FirstArmonic_PhaseC_Value','Vabc_generator_FirstArmonic_PhaseC_Angle'
]
firstArmonic_columns_generator = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','ScaledLength',
'Iabc_generator_FirstArmonic_PhaseA_Value','Iabc_generator_FirstArmonic_PhaseA_Angle','Iabc_generator_FirstArmonic_PhaseB_Value','Iabc_generator_FirstArmonic_PhaseB_Angle','Iabc_generator_FirstArmonic_PhaseC_Value','Iabc_generator_FirstArmonic_PhaseC_Angle',
'Vabc_generator_FirstArmonic_PhaseA_Value','Vabc_generator_FirstArmonic_PhaseA_Angle','Vabc_generator_FirstArmonic_PhaseB_Value','Vabc_generator_FirstArmonic_PhaseB_Angle','Vabc_generator_FirstArmonic_PhaseC_Value','Vabc_generator_FirstArmonic_PhaseC_Angle'
]

firstArmonic_columns_consumator_normal = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','TotalLength',
'Iabc_consumator_FirstArmonic_PhaseA_Value','Iabc_consumator_FirstArmonic_PhaseA_Angle','Iabc_consumator_FirstArmonic_PhaseB_Value','Iabc_consumator_FirstArmonic_PhaseB_Angle','Iabc_consumator_FirstArmonic_PhaseC_Value','Iabc_consumator_FirstArmonic_PhaseC_Angle',
'Vabc_consumator_FirstArmonic_PhaseA_Value','Vabc_consumator_FirstArmonic_PhaseA_Angle','Vabc_consumator_FirstArmonic_PhaseB_Value','Vabc_consumator_FirstArmonic_PhaseB_Angle','Vabc_consumator_FirstArmonic_PhaseC_Value','Vabc_consumator_FirstArmonic_PhaseC_Angle'
]
firstArmonic_columns_consumator = ['FaultLocation_FromGenerator','Is_A','Is_B','Is_C',
'HighVoltage_LongLine500','HighVoltage_ShortLine','MediumVoltage_ShortLine','VoltageGenerator','ScaledLength',
'Iabc_consumator_FirstArmonic_PhaseA_Value','Iabc_consumator_FirstArmonic_PhaseA_Angle','Iabc_consumator_FirstArmonic_PhaseB_Value','Iabc_consumator_FirstArmonic_PhaseB_Angle','Iabc_consumator_FirstArmonic_PhaseC_Value','Iabc_consumator_FirstArmonic_PhaseC_Angle',
'Vabc_consumator_FirstArmonic_PhaseA_Value','Vabc_consumator_FirstArmonic_PhaseA_Angle','Vabc_consumator_FirstArmonic_PhaseB_Value','Vabc_consumator_FirstArmonic_PhaseB_Angle','Vabc_consumator_FirstArmonic_PhaseC_Value','Vabc_consumator_FirstArmonic_PhaseC_Angle'
]

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import load_model
import joblib

In [16]:
_data = scaled_data_normal[scaled_data_normal["cycle"]==10]

In [17]:
x = _data[list(_data.columns)[1:]]
y = _data[list(_data.columns)[0]]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state = 42)

In [20]:
x_test.to_csv("test.csv")

Informații
Regression error matrix
Root mean square error
RMS=√(1/n ∑_(i=0)^n▒(y_i-(y_med ) ̂)^2 )

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

https://scikit-learn.org/stable/modules/sgd.html#regression

https://scikit-learn.org/stable/modules/linear_model.html#lasso

https://scikit-learn.org/stable/modules/linear_model.html#elastic-net

https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression

https://scikit-learn.org/stable/modules/svm.html#regression

https://scikit-learn.org/stable/modules/svm.html#regression

https://scikit-learn.org/stable/modules/ensemble.html

https://u-next.com/blogs/machine-learning/popular-regression-algorithms-ml/

Linear Regression

Ridge Regression

Neural Network Regression 

Lasso Regression 

Decision Tree Regression 

Random Forest

KNN Model 

Support Vector Machines (SVM)

Gausian Regression

Polynomial Regression

## LinearRegression MODEL
pag 68

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [8]:
def ComputeLinearRegression(data_frame, _cycle, columns = studied_columns, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    model = linear_model.LinearRegression()
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    coeff_df = pd.DataFrame(model.coef_,x.columns,columns=['Coefficient'])
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
    outputStr += '\n' + 'COEFFICIENT'+'\n'
    for index in coeff_df.index.to_list():
        outputStr += str(index)+'\t'+str(coeff_df.at[index,'Coefficient']) + '\n'
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeRidgeRegression(data_frame, _cycle, columns = studied_columns, _alpha=1.0, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    model = linear_model.Ridge(alpha=_alpha)
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    coeff_df = pd.DataFrame(model.coef_,x.columns,columns=['Coefficient'])
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
    outputStr += '\n' + 'COEFFICIENT'+'\n'
    for index in coeff_df.index.to_list():
        outputStr += str(index)+'\t'+str(coeff_df.at[index,'Coefficient']) + '\n'
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeRidgeRegressionCV(data_frame, _cycle, columns = studied_columns, _alphas=(0.1, 1.0, 10.0), _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    model = linear_model.RidgeCV(alphas=_alphas)
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    coeff_df = pd.DataFrame(model.coef_,x.columns,columns=['Coefficient'])
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
    outputStr += '\n' + 'COEFFICIENT'+'\n'
    for index in coeff_df.index.to_list():
        outputStr += str(index)+'\t'+str(coeff_df.at[index,'Coefficient']) + '\n'
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeLassoRegression(data_frame, _cycle, columns = studied_columns, _alpha=1.0, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    model = linear_model.Lasso(alpha=_alpha)
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    coeff_df = pd.DataFrame(model.coef_,x.columns,columns=['Coefficient'])
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
    outputStr += '\n' + 'COEFFICIENT'+'\n'
    for index in coeff_df.index.to_list():
        outputStr += str(index)+'\t'+str(coeff_df.at[index,'Coefficient']) + '\n'
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeLassoRegression(data_frame, _cycle, columns = studied_columns, _alpha=1.0, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    model = linear_model.Lasso(alpha = _alpha)
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    coeff_df = pd.DataFrame(model.coef_,x.columns,columns=['Coefficient'])
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
    outputStr += '\n' + 'COEFFICIENT'+'\n'
    for index in coeff_df.index.to_list():
        outputStr += str(index)+'\t'+str(coeff_df.at[index,'Coefficient']) + '\n'
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)
    
def ComputeElasticNet(data_frame, _cycle, columns = studied_columns, _alpha=1.0, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    model = linear_model.ElasticNet(alpha=_alpha)
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    coeff_df = pd.DataFrame(model.coef_,x.columns,columns=['Coefficient'])
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
    outputStr += '\n' + 'COEFFICIENT'+'\n'
    for index in coeff_df.index.to_list():
        outputStr += str(index)+'\t'+str(coeff_df.at[index,'Coefficient']) + '\n'
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeSGDRegressor(data_frame, _cycle, columns = studied_columns, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    model = linear_model.SGDRegressor()
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    coeff_df = pd.DataFrame(model.coef_,x.columns,columns=['Coefficient'])
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
    outputStr += '\n' + 'COEFFICIENT'+'\n'
    for index in coeff_df.index.to_list():
        outputStr += str(index)+'\t'+str(coeff_df.at[index,'Coefficient']) + '\n'
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputePolynomialFeatures(data_frame, _cycle, columns = studied_columns, _degree = 2, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    poly = PolynomialFeatures(degree=_degree, include_bias=False)
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_mod = poly.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x_mod,y,test_size=_test_size, random_state = _random_state)
    
    model = linear_model.LinearRegression()
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeSVRFeatures(data_frame, _cycle, columns = studied_columns, _kernel = 'rbf', _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    
    model = SVR(kernel=_kernel)
    model.fit(x_train,y_train)
    predictions=model.predict(x_test)
    
    outputStr = ''
    outputStr += '\n' + 'model.intercept_'+'\t'+str(model.intercept_)+'\n'
    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(metrics.mean_absolute_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(metrics.mean_squared_error(y_test,predictions))+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(np.sqrt(metrics.mean_squared_error(y_test,predictions)))+'\n'
        
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(predictions[i]) + '\t' + str((list(y_test)[i] - predictions[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

In [9]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [14]:
def ComputeMachineLearning_Columns1_Scaled(data_frame, _cycle, columns = studied_columns, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    
    model = Sequential()
    
    model.add(Dense(40,activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop',
              loss='mse')
    model.fit(x=x_train,y=y_train, validation_data=(x_test,y_test), batch_size=32, epochs=400, verbose=0)
    pred= model.predict(x_test)
    meanAbsoluteError1 = mean_absolute_error(y_test,pred)
    meanSquareError2 = mean_squared_error(y_test,pred)
    meanSquareError3 = mean_squared_error(y_test,pred)**0.5
    
    outputStr = ''

    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(meanAbsoluteError1)+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(meanSquareError2)+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(meanSquareError3)+'\n'
    
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(list(pred)[i]) + '\t' + str((list(y_test)[i] - list(pred)[i])*100/list(y_test)[i]) + '\n'
        
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeMachineLearning_Columns2_Scaled(data_frame, _cycle, columns = studied_columns, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    
    model = Sequential()
    
    model.add(Dense(6,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop',
              loss='mse')
    model.fit(x=x_train,y=y_train, validation_data=(x_test,y_test), batch_size=32, epochs=400, verbose=0)
    pred= model.predict(x_test)
    meanAbsoluteError1 = mean_absolute_error(y_test,pred)
    meanSquareError2 = mean_squared_error(y_test,pred)
    meanSquareError3 = mean_squared_error(y_test,pred)**0.5
    
    outputStr = ''

    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(meanAbsoluteError1)+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(meanSquareError2)+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(meanSquareError3)+'\n'

    
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(list(pred)[i]) + '\t' + str((list(y_test)[i] - list(pred)[i])*100/list(y_test)[i]) + '\n'
        
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeMachineLearning_Columns1_Normal(data_frame, _cycle, columns = studied_columns, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    scaler=MinMaxScaler()
    x_train=scaler.fit_transform(x_train)
    x_test=scaler.transform(x_test)
    model = Sequential()
    
    model.add(Dense(40,activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop',
              loss='mse')
    model.fit(x=x_train,y=y_train, validation_data=(x_test,y_test), batch_size=32, epochs=400, verbose=0)
    pred= model.predict(x_test)
    meanAbsoluteError1 = mean_absolute_error(y_test,pred)
    meanSquareError2 = mean_squared_error(y_test,pred)
    meanSquareError3 = mean_squared_error(y_test,pred)**0.5
    
    outputStr = ''

    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(meanAbsoluteError1)+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(meanSquareError2)+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(meanSquareError3)+'\n'
    
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(list(pred)[i]) + '\t' + str((list(y_test)[i] - list(pred)[i])*100/list(y_test)[i]) + '\n'
        
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

def ComputeMachineLearning_Columns2_Normal(data_frame, _cycle, columns = studied_columns, _test_size=0.3, _random_state = 42, outpudir="output", outputFile="outputFile"):
    _data = data_frame[data_frame["cycle"] == _cycle][columns]
    x = _data[list(_data.columns)[1:]]
    y = _data[list(_data.columns)[0]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=_test_size, random_state = _random_state)
    scaler=MinMaxScaler()
    x_train=scaler.fit_transform(x_train)
    x_test=scaler.transform(x_test)    
    model = Sequential()
    
    model.add(Dense(6,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop',
              loss='mse')
    model.fit(x=x_train,y=y_train, validation_data=(x_test,y_test), batch_size=32, epochs=400, verbose=0)
    pred= model.predict(x_test)
    meanAbsoluteError1 = mean_absolute_error(y_test,pred)
    meanSquareError2 = mean_squared_error(y_test,pred)
    meanSquareError3 = mean_squared_error(y_test,pred)**0.5
    
    outputStr = ''

    outputStr += 'metrics.mean_absolute_error(y_test,predictions)'+'\t'+str(meanAbsoluteError1)+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(meanSquareError2)+'\n'
    outputStr += 'metrics.mean_squared_error(y_test,predictions)'+'\t'+str(meanSquareError3)+'\n'
    
    outputStr += '\n' + 'i\tValue\tPrediction\tError'+'\n'
    for i in range(len(y_test)):
        outputStr += str(i) + '\t' + str(list(y_test)[i]) + '\t' + str(list(pred)[i]) + '\t' + str((list(y_test)[i] - list(pred)[i])*100/list(y_test)[i]) + '\n'
    isExist = os.path.exists(outpudir)
    if not isExist:
        os.makedirs(outpudir)
    dir_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.txt')
    model_name = os.path.join(os.path.join(os.getcwd(),outpudir),outputFile+'.pkl')
    with open(dir_name,'w') as f:
        f.writelines(outputStr)
    joblib.dump(model, model_name)

In [ ]:
cycleList = list(data_normal["cycle"].unique())[10]
_data = scaled_data_normal
_columns = studied_columns_all
_outpudir = "studied_columns_all"

In [ ]:
ComputeMachineLearning_Columns1_Scaled(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)

In [15]:
_k = 'linear'
cycleList = list(data_normal["cycle"].unique())[10:]

In [16]:
cycleList = list(data_normal["cycle"].unique())[10:]
_data = scaled_data_normal
_columns = studied_columns_all
_outpudir = "studied_columns_all"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns1_Scaled(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)
_columns = studied_columns_generator
_outpudir = "studied_columns_generator"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns1_Scaled(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)
_columns = studied_columns_consumator
_outpudir = "studied_columns_consumator"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns1_Scaled(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)

_columns = firstArmonic_columns_all
_outpudir = "firstArmonic_columns_all"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns2_Scaled(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)
_columns = firstArmonic_columns_generator
_outpudir = "firstArmonic_columns_generator"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns2_Scaled(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)
_columns = firstArmonic_columns_consumator
_outpudir = "firstArmonic_columns_consumator"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns2_Scaled(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)

5/5 [==============================] - 0s 999us/step


In [17]:
_data = data_normal
_columns = studied_columns_all_normal
_outpudir = "studied_columns_all"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns1_Normal(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)
_columns = studied_columns_generator_normal
_outpudir = "studied_columns_generator"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns1_Normal(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)
_columns = studied_columns_consumator_normal
_outpudir = "studied_columns_consumator"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns1_Normal(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)

_columns = firstArmonic_columns_all_normal
_outpudir = "firstArmonic_columns_all"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns2_Normal(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)
_columns = firstArmonic_columns_generator_normal
_outpudir = "firstArmonic_columns_generator"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns2_Normal(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)
_columns = firstArmonic_columns_consumator_normal
_outpudir = "firstArmonic_columns_consumator"
for _cycle in cycleList:
    _name = "Cycle_"+str(_cycle)
    ComputeMachineLearning_Columns2_Normal(_data, _cycle, columns = _columns, _test_size=0.3, _random_state = 42, outpudir=_outpudir, outputFile=_name)

5/5 [==============================] - 0s 1ms/step


In [20]:
#pagina 102